In [ ]:
import random
import numpy as np
from statsmodels.tsa.arima_process import arma_generate_sample
import matplotlib.pyplot as plt
import pandas as pd 
import hts
from hts.hierarchy import HierarchyTree

from  forecasting_functions import probabilistic_forecasting
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

from darts.utils.missing_values import fill_missing_values
from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.models import TCNModel,StatsForecastAutoARIMA,NHiTSModel
import scipy.stats as ss
from darts.utils.likelihood_models import GaussianLikelihood

import properscoring as ps

# Fitting ARIMA model manually 
from statsmodels.tsa.arima.model import ARIMA
from sklearn.metrics import mean_absolute_percentage_error
from math import sqrt
from sklearn.metrics import mean_squared_error
import itertools 



In [ ]:
''' pl follow steps from: https://github.com/time-series-foundation-models/lag-llama/blob/main/README.md 
to set the working directory '''

In [ ]:
cd /lag-llama/


In [ ]:
!huggingface-cli download time-series-foundation-models/Lag-Llama lag-llama.ckpt --local-dir  lag-llama

In [ ]:
from itertools import islice

from matplotlib import pyplot as plt
import matplotlib.dates as mdates
from tqdm.autonotebook import tqdm

import torch
from gluonts.evaluation import make_evaluation_predictions, Evaluator
from gluonts.dataset.repository.datasets import get_dataset

from gluonts.dataset.pandas import PandasDataset
import pandas as pd

from lag_llama.gluon.estimator import LagLlamaEstimator

In [ ]:
def get_lag_llama_predictions(dataset, prediction_length, context_length=32, num_samples=20, device="cuda", batch_size=64, nonnegative_pred_samples=True):
    ckpt = torch.load("lag-llama.ckpt", map_location=device)
    estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

    estimator = LagLlamaEstimator(
        ckpt_path="lag-llama.ckpt",
        prediction_length=prediction_length,
        context_length=context_length,

        # estimator args
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        scaling=estimator_args["scaling"],
        time_feat=estimator_args["time_feat"],

        nonnegative_pred_samples=nonnegative_pred_samples,

        # linear positional encoding scaling
        rope_scaling={
            "type": "linear",
            "factor": max(1.0, (context_length + prediction_length) / estimator_args["context_length"]),
        },

        batch_size=batch_size,
        num_parallel_samples=num_samples,
    )

    lightning_module = estimator.create_lightning_module()
    transformation = estimator.create_transformation()
    predictor = estimator.create_predictor(transformation, lightning_module)

    forecast_it, ts_it = make_evaluation_predictions(
        dataset=dataset,
        predictor=predictor,
        num_samples=num_samples
    )
    forecasts = list(tqdm(forecast_it, total=len(dataset), desc="Forecasting batches"))
    tss = list(tqdm(ts_it, total=len(dataset), desc="Ground truth"))

    return forecasts, tss

In [ ]:
'''  We have a hierarchy with a total number of series n = 7, aggregated series
r = 3 and bottom series m = 4. Each series at the bottom level is generated
using the ARIMA processes with linear trend. Parameters p and q are selected randomly from {0,1,2}.
Error terms for the bottom-level series are drawn from multivariate Gaussian. We consider
 the strong correlation between the series with the same parent and the moderate or small correlation
 between the series with different parents.

We have folowing hieararchical structure
           total   - level 0
          /   \
         A     B   - level 1
       / \    / \
      AA AB BA BB  - level 2 or bottom level
      
      '''
def make_arma(num):
    '''Function generates the samples from ARIMA process
    
    parameter
    -----------
    num : number of sample points we want to generate
    
    '''
    p_q_par = [0,1,2]
    
    # specify the ARIMA parameters
    p = random.sample(p_q_par,1)[0]  # order of the autoregressive component
    q = random.sample(p_q_par,1)[0]  # order of the moving average component
    if p ==  0 and q == 1 :
        arparams = np.array([0])
        maparams = np.array([0.7])
        arparams = np.r_[1, -arparams]
        maparams = np.r_[1, maparams]
        y = arma_generate_sample(arparams, maparams, num)
        return(y)
    elif p ==  1 and q == 0 :
        arparams = np.array([0.4])
        maparams = np.array([0])
        arparams = np.r_[1, -arparams]
        maparams = np.r_[1, maparams]
        y = arma_generate_sample(arparams, maparams, num)
        return(y)
    elif p ==  1 and q == 1 :
        arparams = np.array([0.4])
        maparams = np.array([0.7])
        arparams = np.r_[1, -arparams]
        maparams = np.r_[1, maparams]
        y = arma_generate_sample(arparams, maparams, num)
        return(y)
    elif p ==  2 and q == 1 :
        arparams = np.array([0.4,0.2])
        maparams = np.array([0.7])
        arparams = np.r_[1, -arparams]
        maparams = np.r_[1, maparams]
        y = arma_generate_sample(arparams, maparams, num)
        return(y)
    elif p == 1 and q == 2 :
        arparams = np.array([0.4])
        maparams = np.array([0.7,0.3])
        arparams = np.r_[1, -arparams]
        maparams = np.r_[1, maparams]
        y = arma_generate_sample(arparams, maparams, num)
        return(y)
    else  :
        arparams = np.array([0.4,0.2])
        maparams = np.array([0.7,0.3])
        arparams = np.r_[1, -arparams]
        maparams = np.r_[1, maparams]
        y = arma_generate_sample(arparams, maparams, num)
        return(y)



In [ ]:
# Here we set the length of the train, test and validation set 
# We can generate the time series of desired size using following parameters
np.random.seed(1001)

try:
    test_len = int(input("Please enter the test length as an integer: "))
    print(f"Test length is set to: {test_len}")
except ValueError:
    print("Invalid input. Please enter a valid integer.")

train_len = 300
total_len = train_len+test_len
split_val = train_len



''' We took the 110 samples from Arima process last 10 observation as test points'''
AA = make_arma(total_len) 
AB = make_arma(total_len)
BA = make_arma(total_len)
BB = make_arma(total_len)


'''Defining mean and covariance matrix to draw residuals from mutlivariate gaussian'''
mean = [5,10,7,8]


cov = [[2,0.8,0.5,0.2], [0.8, 3,0.22,0.23],[0.5,0.22,3,0.7],[0.2,0.23,0.7,3]]

AA0,AB0,BA0,BB0  = np.random.multivariate_normal(mean, cov, total_len).T

# Adding residuals to the samples 
AA = np.add(AA0,AA)
AB = np.add(AB0,AB)
BA = np.add(BA0,BA)
BB = np.add(BB0,BB)

# defining linear trend for each series

x = np.linspace(0,3,total_len)

AA1 = 10.5*x/2
AB1 = 11*x/2
BA1 = 13.5*x/2
BB1 = 13*x/2
# Adding trend to the series 
AA = np.add(AA1,AA)
AB = np.add(AB1,AB)
BA = np.add(BA1,BA)
BB = np.add(BB1,BB)

''' Defining aggregated series 'total, 'A', 'B' where aggregated series 'A' is the 
sum of series 'AA' and 'AB' and series 'B' is the sum of series 'BA' and 'BB' and the 
most aggregated series 'total' is the sum of 'A' and 'B'  '''
A = np.add(AA,AB)
B = np.add(BA,BB)
total = np.add(A,B)

In [ ]:
# plotting hierarchical data 
plt.plot(total, label = 'Total') 
plt.plot(A, label = 'A')
plt.plot(B, label = 'B')
plt.plot(AA, label = 'AA')
plt.plot(AB, label = 'AB')
plt.plot(BA, label = 'BA')
plt.plot(BB, label = 'BB')
plt.legend()
plt.title('Time series plot for hierarchical data')
plt.show()


In [ ]:
#creating data frame and adding dates to the data for timse series forecast 
data = {'total': total,'A':A,'B':B, 'AA': AA, 'AB': AB, 'BA':BA,'BB':BB}
df = pd.DataFrame(data)

date = pd.date_range(start='2022/1/1', periods=total_len)
df.insert(0,'date',date)
df = df.set_index('date')
df11  = df


In [ ]:

''' To use the Darts model, we need to transform the data into the form mentoned in their documentation.
 First, we create the levels according to the hierarchical structure and then 
 define the hierarchy.    
 '''
#levels and hierarchy will vary from data to data 

level0 = [0]        # level 0
level1 = np.array(df.columns[1:3])  # level 1
level1_r = list(range(1,3))    #stored the indices of the series 
level2 = np.array(df.columns[3:7])  # bottom level 
level2_r = list(range(3,7))                    
levels = [level0,level1_r,level2_r]
bottom_level = level2_r
levels1 = [level1,level2]

In [ ]:
# Defining hierarchy

total = {'total' : list(level1)}
level1_h = {k: [v for v in level2  if v.startswith(k)] for k in level1}

hierarchy = {**total,**level1_h}

aac = list(hierarchy.keys())   # aggregated series 
aab= list(hierarchy.values())  # child nodes of each aggregated series 

In [ ]:
'''Defining aggregation structure that will be needed while geeting the aggregated
forecast using child node forecasts'''
agg_structure1 = []
for i in range(len(aac)):
    for j in range(len(levels1)):
        if aab[i][0] in levels1[j]:
            abcd = []
            for l in aab[i]:
                abcd.append(list(levels1[j]).index(l))
            agg_structure1.append(abcd)


'''Adding bottom level to the aggregation structure'''
for i in range(len(bottom_level)):
    agg_structure1.append([i])
    

In [ ]:
#defining the tree for hierarchy 
ht = HierarchyTree.from_nodes(nodes=hierarchy, df=df)
sum_mat, sum_mat_labels = hts.functions.to_sum_mat(ht) #sum_mat_lables are the all the nodes in the hierarchy
df2 = pd.DataFrame(columns = sum_mat_labels)
for col in sum_mat_labels:
    df2[col] = df[col]
   
#defining the dataframe according to the hierarchy
for col in sum_mat_labels:
    df2[col] = df[col]
    
df2 = df.iloc[: , :len(agg_structure1)]
df2['date'] =  df2.index
df =df2
df = df.drop(df.columns[-1], axis=1)
df_test = df[train_len: len(df2)] 
columns = df.columns

In [ ]:
df_list = []        # origignal data in list format 
for i in columns:
    df_list.append(df2[i].to_list())

In [ ]:
from gluonts.dataset.common import ListDataset

In [ ]:
metadata = {
    'prediction_length': test_len,
    'freq': '1D'
}


train_data = [{"start": df11.index[0], "target": df11[i].values[:-metadata['prediction_length']]} for i in df11.columns]
test_data = [{"start": df11.index[0], "target": df11[i].values} for i in df11.columns]

train_ds = ListDataset(
    data_iter=train_data,
    freq=metadata['freq']
)

test_ds = ListDataset(
    data_iter=test_data,
    freq=metadata['freq']
)
device = torch.device("cuda:0")

In [ ]:
from gluonts.mx import SimpleFeedForwardEstimator, Trainer

In [ ]:
#metadata
prediction_length = test_len
context_length = prediction_length
num_samples = 60
device = "cuda"

In [ ]:
forecasts, tss = get_lag_llama_predictions(
    test_ds,
    prediction_length=prediction_length,
    num_samples=num_samples,
    context_length=context_length,
    device=device
)

In [ ]:
plt.figure(figsize=(20, 15))
date_formater = mdates.DateFormatter('%b, %d')
plt.rcParams.update({'font.size': 15})

# Iterate through the first 9 series, and plot the predicted samples
for idx, (forecast, ts) in islice(enumerate(zip(forecasts, tss)), 9):
    ax = plt.subplot(3, 3, idx+1)

    plt.plot(ts[-4 * prediction_length:].to_timestamp(), label="target", )
    forecast.plot( color='g')
    plt.xticks(rotation=60)
    ax.xaxis.set_major_formatter(date_formater)
    ax.set_title(forecast.item_id)

plt.gcf().tight_layout()
plt.legend()
plt.show()

In [ ]:
from gluonts.evaluation import Evaluator
quantile = [0.1, 0.2, 0.3, 0.4,0.5, 0.6, 0.7, 0.8, 0.9]
evaluator = Evaluator(quantile)
agg_metrics, item_metrics = evaluator(tss, forecasts)
agg_metrics

In [ ]:
ckpt = torch.load("lag-llama.ckpt", map_location=device)
estimator_args = ckpt["hyper_parameters"]["model_kwargs"]

estimator = LagLlamaEstimator(
        ckpt_path="lag-llama.ckpt",
        prediction_length=prediction_length,
        context_length=context_length,

        # distr_output="neg_bin",
        # scaling="mean",
        nonnegative_pred_samples=True,
        aug_prob=0,
        lr=5e-3,

        # estimator args
        input_size=estimator_args["input_size"],
        n_layer=estimator_args["n_layer"],
        n_embd_per_head=estimator_args["n_embd_per_head"],
        n_head=estimator_args["n_head"],
        time_feat=estimator_args["time_feat"],

        # rope_scaling={
        #     "type": "linear",
        #     "factor": max(1.0, (context_length + prediction_length) / estimator_args["context_length"]),
        # },

        batch_size=16,
        num_parallel_samples=num_samples,
        trainer_kwargs = {"max_epochs": 150,}, # <- lightning trainer arguments
    )

In [ ]:
from gluonts.evaluation import make_evaluation_predictions

In [ ]:
predictor = estimator.train(train_ds, cache_data=True, shuffle_buffer_length=1000)

In [ ]:
forecast_it, ts_it = make_evaluation_predictions(
        dataset=test_ds,
        predictor=predictor,
        num_samples=num_samples
    )


forecasts =  list(tqdm(forecast_it, desc="Forecasting batches"))
tss =  list(tqdm(ts_it,desc="Ground truth"))

In [ ]:
from gluonts.evaluation import Evaluator
evaluator = Evaluator(quantile)
agg_metrics, item_metrics = evaluator(tss, forecasts)
agg_metrics

In [ ]:
plt.figure(figsize=(20, 15))
date_formater = mdates.DateFormatter('%b, %d')
plt.rcParams.update({'font.size': 15})

# Iterate through the first 9 series, and plot the predicted samples
for idx, (forecast, ts) in islice(enumerate(zip(forecasts, tss)), 9):
    ax = plt.subplot(3, 3, idx+1)

    plt.plot(ts[-4 * prediction_length:].to_timestamp(), label="target", )
    forecast.plot( color='g')
    plt.xticks(rotation=60)
    ax.xaxis.set_major_formatter(date_formater)
    ax.set_title(forecast.item_id)

plt.gcf().tight_layout()
plt.legend()
plt.show()

In [ ]:

def process_forecasts(forecasts):
    samples1 = [forecast.samples for forecast in forecasts]
    samples = np.transpose(samples1, (0, 2, 1))
    samples = np.array(samples)
    return samples

# Usage example:
samples = process_forecasts(forecasts)


In [ ]:
metadata['prediction_length']

In [ ]:
import numpy as np


a = num_samples
hist_i = a // metadata['prediction_length']

past_m = []   
past_std = []

for i in range(hist_i):
    try:
        train_data_h = [{"start": df11.index[0], "target": df11[i1].values[:-a ]} for i1 in df11.columns]
        train_ds_h = ListDataset(data_iter=train_data_h, freq=metadata['freq'])
        forecast_it_h, ts_it_h = make_evaluation_predictions(
            dataset=train_ds_h,
            predictor=predictor,
            num_samples=num_samples)

        forecasts_h = list(tqdm(forecast_it_h, desc="Forecasting batches"))
        tss_h = list(tqdm(ts_it_h, desc="Ground truth"))
    
        samples11 = []
        for n in range(len(forecasts_h)):
            samples11.append(forecasts_h[n].samples)
            
        samples1 = np.transpose(samples11, (0, 2, 1))
        samples1 = samples1.tolist()
        samples1 = np.array(samples1)

        for k in range(len(samples1)):
            aa = np.mean(samples1[k], axis=1)
            ab = np.std(samples1[k], axis=1)
            
            if i <= 0:
                past_m.append(aa)
                past_std.append(ab)
            else:
                past_m[k] = np.hstack((past_m[k], aa))
                past_std[k] = np.hstack((past_std[k], ab))

        a = a - metadata['prediction_length']

    except Exception as e:
        print(f"An error occurred: {e}")
        # Handle the error gracefully, log it, and possibly retry or exit the loop


In [ ]:
'''Defining the standard residuals for the past data., taking mean and sd from past data samples 

     residuals = (actual-mean(predicted))/std(predicted)
     
 We want to store the ranks of these residuals to use in revised bottom-up forecasting. '''
res = []
for i in range(len(samples)):
    ab = []
    for j in range(num_samples):
        p = len(df[columns[0]])-num_samples-test_len
        aa = ((df[columns[i]][p+j]-past_m[i][j]))/past_std[i][j]
        ab.append(aa)
    res.append(ab)

In [ ]:
'''We need sorted samples to re-order accoeding to the ranks of the residuals'''

prob_forecasting = probabilistic_forecasting()

sorted_samples = prob_forecasting.sample_sorting_1(samples) # sample_sorting function is defined in forecasting_funtions.py file 


In [ ]:
#defining ranks for the residuals
ranks=[]
for i in range(len(res)):
    ranks.append(ss.rankdata(res[i]))

In [ ]:
reorderd_samples = prob_forecasting.sample_reordering_3(samples, ranks)

In [ ]:

BU_len = sum_mat.shape[1]       #len of the bottom series 
agg_len = len(sum_mat_labels)-BU_len  #length of the aggregated series
agg_samples_index = list(range(0, agg_len))  #indices of the aggregated series 
agg_series_ = columns[0:agg_len]           # names of the aggregated series 

bottom_series = prob_forecasting.getting_bottom_series(samples, sum_mat,sum_mat_labels)   #bottom level without reodering
bottom_series_r = prob_forecasting.getting_bottom_series(reorderd_samples,sum_mat,sum_mat_labels) #bottom series with rank reodering

aggre_series_numbers = agg_structure1     #aggregation structutre 


In [ ]:
# bottom series of original observation 
bottom_series_og = prob_forecasting.getting_bottom_series(df_list,sum_mat,sum_mat_labels) #function defined in forecasting_funtions.py file 


In [ ]:
# forecat using bottom-up method 
bottom_up_samples_ = prob_forecasting.bottom_up_forecast(columns,bottom_series,bottom_level,aggre_series_numbers,levels) #bottom_up_forecast function defined in forecasting_funtions.py file
bottom_up_samples_ = prob_forecasting.list_reversal(bottom_up_samples_) # list_reversal function defined in forecasting_funtions.py file

In [ ]:
# BU forecast with reodering, we reoder the forecasted samples according to the ranks of the past forecast residuals
revised_samples_ = prob_forecasting.bottom_up_revised_forecast(columns,bottom_series_r,bottom_level,aggre_series_numbers,levels,ranks)
revised_samples_ = prob_forecasting.list_reversal(revised_samples_)

In [ ]:
#Bottom up forecast using outlier reordering technique
bottom_up_forecast_h = prob_forecasting.bottom_up_forecast_hueristic(bottom_series, bottom_series_og,columns,bottom_level,aggre_series_numbers,levels,0.1)
bottom_up_samples_h = prob_forecasting.list_reversal(bottom_up_forecast_h)

In [ ]:
#mean forecast using all three methods 
mean_bottom_up_ = prob_forecasting.finding_mean(bottom_up_samples_)
mean_revised_forecast_ = prob_forecasting.finding_mean(revised_samples_)
mean_bottom_up_h = prob_forecasting.finding_mean(bottom_up_samples_h)

In [ ]:
from pandas import Period
# Define the start date
start_period = Period(df_test.index[0], 'D')

In [ ]:
from gluonts.model.forecast import SampleForecast

bottom_up_samples_ = np.transpose(bottom_up_samples_,(0,2,1))

revised_samples_ = np.transpose(revised_samples_,(0,2,1))

bottom_up_samples_h = np.transpose(bottom_up_samples_h,(0,2,1))

# BU samples 
BU_forecast = [SampleForecast(samples=samples, start_date=start_period) for samples in bottom_up_samples_]


# revised samples
r_forecast = [SampleForecast(samples=samples, start_date=start_period) for samples in revised_samples_]



# HRA samples
h_forecast = [SampleForecast(samples=samples, start_date=start_period) for samples in bottom_up_samples_h]

loss_df = pd.DataFrame(columns=['Method', 'mean_absolute_QuantileLoss', 'mean_wQuantileLoss'])


In [ ]:
from gluonts.evaluation import Evaluator
evaluator = Evaluator(quantile)
agg_metrics, item_metrics = evaluator(tss, forecasts)
new_row = pd.Series({'Method': 'Base_forecast', 'mean_absolute_QuantileLoss': agg_metrics['mean_absolute_QuantileLoss'], 'mean_wQuantileLoss': agg_metrics['mean_wQuantileLoss']})
loss_df.loc[len(loss_df)] = new_row

In [ ]:
evaluator = Evaluator(quantile)
agg_metrics, item_metrics = evaluator(tss, BU_forecast)
new_row = pd.Series({'Method': 'BU_forecast', 'mean_absolute_QuantileLoss': agg_metrics['mean_absolute_QuantileLoss'], 'mean_wQuantileLoss': agg_metrics['mean_wQuantileLoss']})
loss_df.loc[len(loss_df)] = new_row

In [ ]:
evaluator = Evaluator(quantile)
agg_metrics, item_metrics = evaluator(tss, r_forecast)
new_row = pd.Series({'Method': 'r_forecast', 'mean_absolute_QuantileLoss': agg_metrics['mean_absolute_QuantileLoss'], 'mean_wQuantileLoss': agg_metrics['mean_wQuantileLoss']})
loss_df.loc[len(loss_df)] = new_row

In [ ]:
evaluator = Evaluator(quantile)
agg_metrics, item_metrics = evaluator(tss, h_forecast)
new_row = pd.Series({'Method': 'h_forecast', 'mean_absolute_QuantileLoss': agg_metrics['mean_absolute_QuantileLoss'], 'mean_wQuantileLoss': agg_metrics['mean_wQuantileLoss']})
loss_df.loc[len(loss_df)] = new_row

In [ ]:
loss_df
